In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
fifa_20 = pd.read_csv('players_20.csv')
fifa_19 = pd.read_csv('players_19.csv')
fifa_18 = pd.read_csv('players_18.csv')
fifa_17 = pd.read_csv('players_17.csv')
fifa_16 = pd.read_csv('players_16.csv')

In [82]:
fifa_20['year'] = 20
fifa_19['year'] = 19
fifa_18['year'] = 18
fifa_17['year'] = 17
fifa_16['year'] = 16

In [83]:
fifa_20.shape, fifa_19.shape, fifa_18.shape, fifa_17.shape, fifa_16.shape

((18278, 105), (17770, 105), (17592, 105), (17009, 105), (14881, 105))

In [11]:
list(fifa.columns)

['sofifa_id',
 'player_url',
 'short_name',
 'long_name',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'nationality',
 'club',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'player_positions',
 'preferred_foot',
 'international_reputation',
 'weak_foot',
 'skill_moves',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'player_tags',
 'team_position',
 'team_jersey_number',
 'loaned_from',
 'joined',
 'contract_valid_until',
 'nation_position',
 'nation_jersey_number',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_reflexes',
 'gk_speed',
 'gk_positioning',
 'player_traits',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_re

In [84]:
fifa = pd.concat([fifa_20, fifa_19, fifa_18, fifa_17, fifa_16])

In [85]:
fifa.shape

(85530, 105)

In [51]:
leagues_consider = ['English Premier League', 'German 1. Bundesliga', 'French Ligue 1', 'Spain Primera Division', 'Italian Serie A']

In [12]:
team_and_league = pd.read_csv('teams_and_leagues.csv')

In [13]:
team_and_league.head()

,url,league_name
0,241,Spain Primera Division
1,11,English Premier League
2,5,English Premier League
3,243,Spain Primera Division
4,1,English Premier League


In [21]:
team_and_league.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          1080 non-null   int64 
 1   league_name  1080 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.0+ KB


In [86]:
fifa['club'].nunique()

892

In [23]:
fifa['short_name'].nunique()

31584

In [39]:
team_and_league[team_and_league['url']==113173]

,url,league_name
652,113173,Swedish Allsvenskan


In [48]:
!pip install tqdm

In [50]:
clubs = []

# importing the modules
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
for i in tqdm(list(team_and_league['url'].unique())):
    # target url
    url = f'https://sofifa.com/team/{i}/'

    # making requests instance
    reqs = requests.get(url)

    # using the BeautifulSoup module
    soup = BeautifulSoup(reqs.text, 'html.parser')


    for title in soup.find_all('title'):
        titl = title.get_text()
    club_name = titl[:titl.find('FIFA 22')-1]
    clubs.append(club_name)

100%|██████████████████████████████████████████████████████████████████████████████| 1009/1009 [14:02<00:00,  1.20it/s]


In [63]:
#     url = f'https://sofifa.com/team/1/'

#     # making requests instance
#     reqs = requests.get(url)

#     # using the BeautifulSoup module
#     soup = BeautifulSoup(reqs.text, 'html.parser')


#     for title in soup.find_all('title'):
#         titl = title.get_text()
#     print(titl)

In [52]:
url_club = {}
for i, j in zip(list(team_and_league['url'].unique()), clubs):
    url_club[i] = j

In [54]:
clubs = []
for i in team_and_league['url']:
    clubs.append(url_club[i])

team_and_league['club'] = clubs

In [55]:
team_and_league.head()

,url,league_name,club
0,241,Spain Primera Division,FC Barcelona
1,11,English Premier League,Manchester United
2,5,English Premier League,Chelsea
3,243,Spain Primera Division,Real Madrid CF
4,1,English Premier League,Arsenal


In [69]:
team_and_league['league_name'] = team_and_league['league_name'].str.rstrip()

In [71]:
teams_consider = team_and_league[team_and_league['league_name'].isin(leagues_consider)]
teams_consider.reset_index(drop=True, inplace=True)

In [72]:
teams_consider.shape

(147, 3)

In [73]:
teams_consider['league_name'].unique()

array(['Spain Primera Division', 'English Premier League',
       'German 1. Bundesliga', 'French Ligue 1', 'Italian Serie A'],
      dtype=object)

In [76]:
clubs_consider = teams_consider['club'].unique()

In [109]:
fifa_df = fifa[fifa['club'].isin(clubs_consider)]

In [88]:
fifa.shape, fifa_df.shape

((85530, 105), (11781, 105))

In [90]:
features = list(fifa_df.columns)

In [198]:
rem = ['sofifa_id', 'player_url', 'short_name', 'long_name', 'dob', 'nationality', 'player_positions', 'preferred_foot','wage_eur', 'player_tags','work_rate','team_position','nation_position','nation_jersey_number','player_traits','contract_valid_until','loaned_from', 'joined','real_face']

In [199]:
final_feats = list(set(features).difference(set(rem)))

In [200]:
len(final_feats)

86

In [201]:
fifa_final = fifa_df[final_feats]

In [202]:
fifa_final.shape

(11781, 86)

In [203]:
fifa_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11781 entries, 0 to 14859
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   attacking_volleys           11781 non-null  object 
 1   pace                        10473 non-null  float64
 2   movement_acceleration       11781 non-null  object 
 3   st                          10473 non-null  object 
 4   attacking_short_passing     11781 non-null  object 
 5   mentality_vision            11781 non-null  object 
 6   mentality_penalties         11781 non-null  object 
 7   skill_ball_control          11781 non-null  object 
 8   rdm                         10473 non-null  object 
 9   movement_sprint_speed       11781 non-null  object 
 10  team_jersey_number          11781 non-null  float64
 11  defending                   10473 non-null  float64
 12  skill_dribbling             11781 non-null  object 
 13  gk_handling                 130

In [204]:
fifa_final = fifa_final[fifa_final['body_type'].isin(['Normal', 'Lean', 'Stocky'])]

In [205]:
fifa_final.shape

(11762, 86)

In [206]:
rep = {'Normal':1, 'Lean':0, 'Stocky':2}
fifa_final.replace(rep, inplace=True)

In [212]:
fifa_final_df = fifa_final.fillna(0)

In [213]:
fifa_final_df.shape

(11762, 86)

In [214]:
fifa_final_df.groupby('year')['club'].nunique()

year
16    77
17    78
18    79
19    81
20    81
Name: club, dtype: int64

In [215]:
def add_up(x):
    if '-' in str(x):
        a = str(x).split('-')
        a = list(map(float, a))
        return a[0]-a[1]
    a = str(x).split('+')
    a = list(map(float, a))
    return sum(a)

In [216]:
for i in fifa_final_df.columns:
    if i not in ['year', 'club']:
        fifa_final_df[i] = fifa_final_df[i].apply(add_up)

In [217]:
fifa_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11762 entries, 5 to 14859
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   attacking_volleys           11762 non-null  float64
 1   pace                        11762 non-null  float64
 2   movement_acceleration       11762 non-null  float64
 3   st                          11762 non-null  float64
 4   attacking_short_passing     11762 non-null  float64
 5   mentality_vision            11762 non-null  float64
 6   mentality_penalties         11762 non-null  float64
 7   skill_ball_control          11762 non-null  float64
 8   rdm                         11762 non-null  float64
 9   movement_sprint_speed       11762 non-null  float64
 10  team_jersey_number          11762 non-null  float64
 11  defending                   11762 non-null  float64
 12  skill_dribbling             11762 non-null  float64
 13  gk_handling                 117

In [222]:
fifa_grouped = fifa_final_df.groupby(['club', 'year']).mean()

In [224]:
clubs = fifa_final_df['club'].unique()

In [293]:
len(clubs)

81

In [232]:
year = [17, 18, 19, 20]

In [ ]:
new_fifa_diff = pd.DataFrame()

In [248]:

l=[]
for i in clubs:
    for j in year:
        if (i,j) in fifa_grouped.index and (i, j-1) in fifa_grouped.index:
            diff = fifa_grouped.loc[(i,j)] - fifa_grouped.loc[(i, j-1)]
            diff_df = pd.DataFrame(diff).T
            diff_df['club'] = i
#             diff_df['year'] = f'{j}-{j-1}'
            l.append(diff_df)
        else:
            pass

In [255]:
new_diff_df = pd.concat(l)

In [292]:
new_diff_df.shape

(313, 84)

In [257]:
new_diff_df.set_index('club', inplace=True)

In [308]:
len(new_diff_df.loc['Manchester City'].iloc[0].shape)

1

In [312]:
pd.DataFrame(new_diff_df.loc['Manchester City'].mean()).T

,attacking_volleys,pace,movement_acceleration,st,attacking_short_passing,mentality_vision,mentality_penalties,skill_ball_control,rdm,movement_sprint_speed,...,attacking_heading_accuracy,lcb,height_cm,skill_curve,shooting,gk_positioning,physic,lm,mentality_aggression,rwb
0,-0.923459,-1.14629,-0.445402,-0.566092,0.851358,0.373302,-2.348746,-0.109718,0.137931,-0.435737,...,-0.683647,-0.020899,-0.002351,0.192006,-1.022205,0.676071,-1.06139,-0.198276,-0.32419,0.024295


In [327]:
k = []
for i in clubs:
    df = new_diff_df.loc[i]
    if len(df.shape)>1:
        meaned_diff = pd.DataFrame(df.mean(axis=0)).T
    meaned_diff['club'] = i
    k.append(meaned_diff)

In [328]:
final_df = pd.concat(k)

In [329]:
final_df.shape

(81, 85)

In [330]:
final_df['club'].nunique()

80

In [331]:
final_df.set_index('club', inplace=True)
final_df.head()

,attacking_volleys,pace,movement_acceleration,st,attacking_short_passing,mentality_vision,mentality_penalties,skill_ball_control,rdm,movement_sprint_speed,...,attacking_heading_accuracy,lcb,height_cm,skill_curve,shooting,gk_positioning,physic,lm,mentality_aggression,rwb
club,,,,,,,,,,,,,,,,,,,,,
Manchester City,-0.923459,-1.146290,-0.445402,-0.566092,0.851358,0.373302,-2.348746,-0.109718,0.137931,-0.435737,...,-0.683647,-0.020899,-0.002351,0.192006,-1.022205,0.676071,-1.061390,-0.198276,-0.324190,0.024295
FC Barcelona,-0.312211,-0.440104,-0.934896,0.396701,0.121817,-0.023438,-0.355613,0.117766,0.730613,-0.694734,...,0.435185,0.720197,0.303530,-0.766493,-0.290509,-0.232350,0.016493,0.620949,-0.037905,0.589120
Liverpool,-0.482975,-1.371864,-0.873955,-0.675926,-0.989546,0.750299,-0.901135,-0.777180,-0.224014,-0.768220,...,-0.953405,-0.170550,0.183094,-0.193847,-0.895161,0.956989,-1.390084,-0.545699,0.629331,-0.322581
Paris Saint-Germain,-1.098214,0.717634,0.866071,-0.054688,-0.132812,-1.433036,-0.694196,-0.017857,0.453125,0.862723,...,0.164062,0.463170,0.210938,0.622768,-0.960938,-0.581473,-0.414062,0.263393,-0.871652,0.561384
Napoli,-0.600186,-0.176489,-0.339020,0.246898,0.012097,-0.978288,-0.945409,-0.249380,0.463710,-0.727667,...,-0.058623,0.318238,-0.645782,-0.353908,-0.267370,-0.292494,-1.113834,0.455645,-1.035050,0.442618


In [393]:
final_scores = pd.DataFrame()
for i in final_df.columns:
    means = final_df[i]
    max_ = np.max(means)
    min_ = np.min(means)
    temp=[]
    for j in means:
        temp.append(((j-min_)*100)/(max_-min_))
    final_scores[i] = temp
final_scores['club'] = final_df.index

In [394]:
final_scores.head()

,attacking_volleys,pace,movement_acceleration,st,attacking_short_passing,mentality_vision,mentality_penalties,skill_ball_control,rdm,movement_sprint_speed,...,lcb,height_cm,skill_curve,shooting,gk_positioning,physic,lm,mentality_aggression,rwb,club
0,20.334623,34.902748,40.292465,13.911089,50.702016,38.164913,3.457040,25.459886,32.889849,43.809922,...,31.360209,38.804930,40.551319,17.120697,60.883043,19.059854,14.706375,48.870098,28.072430,Manchester City
1,30.271991,50.683321,29.791872,36.163837,35.370114,31.150377,44.541494,29.368684,51.629672,39.132076,...,50.661663,53.381217,23.749188,30.530620,31.522663,50.537021,34.880956,54.314115,47.012387,FC Barcelona
2,27.495790,29.862044,31.099180,11.372530,12.013917,44.830387,33.296655,13.991058,21.445621,37.804816,...,27.462626,47.642043,33.787456,19.449048,69.962389,9.461081,6.150588,67.002321,16.440870,Liverpool
3,17.493533,76.554355,68.426140,25.731024,30.018866,6.228034,37.562281,27.038304,42.855860,67.261907,...,43.967522,48.968863,48.102407,18.243555,20.238899,37.963605,26.075625,38.459531,46.082318,Paris Saint-Germain
4,25.590229,56.574119,42.574577,32.701483,33.064252,14.268185,32.384026,23.060116,43.190533,38.537247,...,40.192856,8.143258,30.981645,30.954702,29.578786,17.528344,30.810112,35.352346,42.099808,Napoli


In [395]:
final_scores.set_index('club', inplace=True)

In [396]:
total_score = pd.DataFrame(final_scores.mean(axis=1), columns=['score'])

In [397]:
total_score.head()

,score
club,
Manchester City,36.310527
FC Barcelona,38.662883
Liverpool,30.541267
Paris Saint-Germain,34.342185
Napoli,35.906827


In [398]:
total_score.sort_values(by='score', ascending=False, inplace=True)

In [399]:
total_score.reset_index(inplace=True)

In [400]:
total_score.head()

,club,score
0,Amiens SC,75.472959
1,SC Paderborn 07,68.160680
2,Parma,67.817000
3,Cagliari,58.292728
4,SD Eibar,57.392832


In [401]:
total_score.to_csv('scores.csv', index=False)

In [402]:
ranks = total_score.reset_index()

In [403]:
ranks.drop('score', axis=1, inplace=True)

In [404]:
ranks.columns=['rank', 'club']
ranks['rank'] = ranks['rank']+1
ranks = ranks[['club', 'rank']]

In [405]:
ranks.head()

,club,rank
0,Amiens SC,1
1,SC Paderborn 07,2
2,Parma,3
3,Cagliari,4
4,SD Eibar,5


In [390]:
# ranks.to_csv('fifa_ranks.csv', index=False)